# antlr

Thus begins the day that my notebook frameworks starts absorbing other frameworks.



## introduction



Why do this. Ultimately, it creates a shortcut for including interpreters for any language in these notebooks. Less reliance on other code, and also converting a library from one framework to another.



In [ ]:
// placeholder for readme.md?


## multilungual abstract syntax trees

This is great day for translating between languages.


### select antlr tree

Using antlr templates because look at all the glorious grammars posted right there on it's main Github page! https://github.com/antlr/grammars-v4



#### the code

select antlr tree?


In [ ]:
var fs = require('fs');
var {CommonTokenStream, ANTLRInputStream} = require('antlr4ts');
var {AbstractParseTreeVisitor} = require('antlr4ts/tree');
var importer = require('../Core')
var {
    selectDom,
    evaluateDom,
    treeToHtml,
    htmlToTree
} = importer.import(['select tree', 'tree to html', 'html to tree'])
var getParser = importer.import('get antlr tool')
var extToLang = importer.import('ext to lang')
var getGenericVisitor = importer.import('antlr tree visitor')

//TODO: fix this with own transpiler because no one is going to share their hard work on StringTemplate4 code
function toString(ctx, subctx) {
    var ast = (subctx || ctx)
    if(typeof ast.ownerDocument !== 'undefined') {
        ast = htmlToTree(ast)
    }
    //escodegen.attachComments(ast, ast.comments, ast.tokens, ast.whitespace)
    return escodegen.generate(ast, {
        comment: true,
        tokens: true,
        whitespace: true,
        format: {indent: {style: '    '}, quotes: 'single'}
    })
}

function selectCode(descriptor, code, language) {
    if(typeof code === 'function') {
        code = code.toString()
    }
    if(typeof code === 'string') {
        if(fs.existsSync(code)) {
            if(typeof language === 'undefined') {
                // TODO: use language file extension from media-server code module gheshi support
                language = extToLang(code)
            }
            code = fs.readFileSync(code).toString('utf8')
        }
        var {lexer, parser, visitor} = getParser(language)
        try {
            const lex = new lexer(new ANTLRInputStream(code))
            const parse = new parser(new CommonTokenStream(lex))
            parse.buildParseTrees = true;
            var {GenericVisitor} = getGenericVisitor({parser, visitor: AbstractParseTreeVisitor})
            var visitor = new GenericVisitor()
            var tree = parse[parser.ruleNames[0]]()
            return visitor.visit(tree)
        } catch(e) {
            console.log(e)
            throw new Error('Could not parse \n' + code)
        }
    }
    // No more selectTree because jsel and friends are inconsistent/partially implemented.
    if(typeof code.evaluate === 'undefined'
       && typeof code.ownerDocument === 'undefined') {
        code = selectDom('//Program', treeToHtml(code))
        code.ownerDocument.toString = toString.bind(code, code)
    }
    return selectDom(descriptor, code)
}

module.exports = {
    selectCode
}


#### test select jison on some quake 3 C code?



In [ ]:
var fs = require('fs')
var importer = require('../Core')
var {selectCode} = importer.import('select antlr tree')

function testAntlr() {
    var result = selectCode('*', '/Users/briancullinan/planet_quake/code/q3_ui/ui_cdkey.c', 'cpp')
    console.log(result)
    console.log(result.body[0])
    console.log(result.body[result.body.length-1])
}

module.exports = testAntlr

if(typeof $$ !== 'undefined') {
    $$.async()
    testAntlr()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


## antlr tools

Creating lexer and parser from .g4 files. Converting XPath to trees. Convert antlr Tool classes to nodejs so we can load grammars on the fly.



### get antlr language



#### the code

get antlr language?

get antlr tool?


In [ ]:

function returnLanguageTools({lexer, parser, visitor, listener}) {
    var result = {}
    result['lexer'] = Object.values(lexer)[0]
    result['parser'] = Object.values(parser)[0]
    result['visitor'] = Object.values(visitor)[0]
    result['listener'] = Object.values(listener)[0]
    return result
}

function getParser(language) {
    // TODO: get parser by language lookup
    if(language === 'cpp' || language === 'c++') {
        var lexer = require('../Resources/Parsers/cpp/CPP14Lexer.js')
        var parser = require('../Resources/Parsers/cpp/CPP14Parser.js')
        var visitor = require('../Resources/Parsers/cpp/CPP14Visitor.js')
        var listener = require('../Resources/Parsers/cpp/CPP14Listener.js')
        return returnLanguageTools({
            lexer,
            parser,
            visitor,
            listener
        })
    }
}

module.exports = getParser



### antlr tree visitor



#### the code

antlr tree visitor?


In [ ]:
var {Interval} = require('antlr4ts/misc')

function typeToString(node) {
    if(node === 'object' && node) {
        return Object.getPrototypeOf(node).constructor.name;
    } else {
        return typeof node;
    }
}

function getGenericToken(parser, token) {
    if(!token) return {}
    return {
        type: parser.ruleNames[token._type],
    //    type2: parser.ruleNames[child._stop._type],
        value: token.source.stream.getText(Interval.of(token.start, token.stop)),
        range: [token.start, token.stop]
    }
}

function getGenericVisitor({parser, visitor}) {
    function GenericVisitor() {
    //    visitor.apply(this, arguments)
    }
    GenericVisitor.prototype = Object.create(visitor.prototype)
    GenericVisitor.prototype.constructor = GenericVisitor
    GenericVisitor.prototype.defaultResult = function () {
        return {children: []}
    }
    GenericVisitor.prototype.aggregateResult = (ctx, child) => {
        if(!Array.isArray(ctx)) ctx = ctx.children
        ctx[ctx.length] = child
        return ctx
    }
    GenericVisitor.prototype.visitDeclarationseq = function (ctx) {
        var dec = ctx.declaration()
        var seq = ctx.declarationseq()
        if(!seq) {
            return dec.accept(this)
        }
        return [...seq.accept(this), ...dec.accept(this)]
    }
    GenericVisitor.prototype.visitTranslationunit = function (ctx) {
        return {
            type: 'Program',
        //    body: ctx.declarationseq().accept(this)
            body: ctx.declarationseq().accept(this)
        }
    }
    GenericVisitor.prototype.visitFunctiondefinition = function (ctx) {
        console.log(ctx.attributespecifierseq())
        console.log()
        console.log(ctx.virtspecifierseq())
        return {
            type: 'Function',
            id: ctx.declarator().accept(this),
            params: ctx.declspecifierseq().accept(this),
            body: ctx.functionbody().accept(this),
            returnType: ctx.declspecifierseq().accept(this),
            void: ctx.declspecifierseq().accept(this) === 'void',
        }
    }
    GenericVisitor.prototype.visitDeclarator = function (ctx) {
        var params = ctx.parametersandqualifiers()
        if(params) return params.accept(this)
        return {
            type: 'Identifier',
            name: getGenericToken(parser, ctx._start).value
        }
    }
    /*
    GenericVisitor.prototype.visitPseudodestructorname = function (ctx) {
        //console.log('hit')
        return {
            id: ctx.decltypespecifier().accept(this)
        }
    }
    GenericVisitor.prototype.visitDeclspecifier = function (ctx) {
        console.log(ctx.Constexpr())
        return {
            type: !!ctx.Typedef() ? 'Type': 'Unk',
        //    body: this.visitChildren(ctx.typespecifier())
            typedef: !!ctx.Typedef()
        }
    }
    GenericVisitor.prototype.visitSimpledeclaration = function (ctx) {
        return {
            type: 'Declarator',
            value: getGenericToken(parser, ctx._start).value,
            body: this.visitChildren(ctx)
        }
    }
    */
    GenericVisitor.prototype.visitTerminal = function (node) {
        return getGenericToken(parser, node._symbol).value
    }
    GenericVisitor.prototype.visitChildren = function (ctx) {
    //    console.log(Object.getPrototypeOf(ctx._parent).constructor.name)
        return visitor.prototype.visitChildren.apply(this, [ctx])
    }
    
    return {
        GenericVisitor
    }
}

module.exports = getGenericVisitor


#### TODO: use g4 translator to automatically select entry point, because this is silly




#### TODO: convert antlr tree to json matching esprima format with text locations and tokens lists




In [ ]:
var {TerminalNodeImpl} = require('antlr')


function typeToString(node) {
    if(jsType === 'object' && node) {
        return Object.getPrototypeOf(node).constructor.name;
    } else {
        return typeof node;
    }
}

// TODO: use transpiler to convert a tree from grammer spec to esprima style
function toAntlrTree(parseTree) {
    if(parseTree instanceof TerminalNodeImpl) {
        
    } else if (parseTree instanceof CommonToken) {
        
    } else if (parseTree instanceof CompilationUnitContext) {
        var result = {}
        result['type'] = 'Program' //typeToString(parseTree)
        result['range'] = [0, parseTree.EOF().getSymbol().start]
        var children = []
        for(var i = 0; i < parseTree.getChildCount(); i++) {
            children.push(toJson(parseTree.getChild(i)))
        }
        result['body'] = children
        return result
    }
}

/*
public class Examples {

  private static final Gson PRETTY_PRINT_GSON = new GsonBuilder().setPrettyPrinting().create();
  private static final Gson GSON = new Gson();

  public static String toJson(ParseTree tree) {
    return toJson(tree, true);
  }

  public static String toJson(ParseTree tree, boolean prettyPrint) {
    return prettyPrint ? PRETTY_PRINT_GSON.toJson(toMap(tree)) : GSON.toJson(toMap(tree));
  }

  public static Map<String, Object> toMap(ParseTree tree) {
    Map<String, Object> map = new LinkedHashMap<>();
    traverse(tree, map);
    return map;
  }

  public static void traverse(ParseTree tree, Map<String, Object> map) {

    if (tree instanceof TerminalNodeImpl) {
      Token token = ((TerminalNodeImpl) tree).getSymbol();
      map.put("type", token.getType());
      map.put("text", token.getText());
    }
    else {
      List<Map<String, Object>> children = new ArrayList<>();
      String name = tree.getClass().getSimpleName().replaceAll("Context$", "");
      map.put(Character.toLowerCase(name.charAt(0)) + name.substring(1), children);

      for (int i = 0; i < tree.getChildCount(); i++) {
        Map<String, Object> nested = new LinkedHashMap<>();
        children.add(nested);
        traverse(tree.getChild(i), nested);
      }
    }
  }

  public static void main(String[] args) {
    String source = "(1 + 2) * 3";
    ExpressionLexer lexer = new ExpressionLexer(CharStreams.fromString(source));
    ExpressionParser parser = new ExpressionParser(new CommonTokenStream(lexer));
    System.out.println(toJson(parser.parse()));
  }
}
*/

module.export = toAntlrTree


#### lets not forget what an esprima tree looks


In [ ]:
var importer = require('../Core')
var {selectAst} = importer.import('select code tree')
var {htmlToTree} = importer.import('html to tree')

var root = selectAst('//Program', importer.interpret('select antlr tree').code)
console.log(htmlToTree(root))

/*

expects
{
  type: 'Program',
  body: [
    {
      type: 'VariableDeclaration',
      declarations: [Array],
      kind: 'var',
      range: [Array]
    },
    {
      type: 'VariableDeclaration',
      declarations: [Array],
      kind: 'var',
      range: [Array]
    },
  ],
  sourcetype: 'script',
  range: [ 0, 2317 ],
  comments: [
    {
      type: 'Line',
      value: ' TODO: get parser by language lookup',
      range: [Array]
    },
  ],
  tokens: [
    { type: 'Keyword', value: 'var', range: [Array] },
    { type: 'Identifier', value: 'fs', range: [Array] },
    { type: 'Punctuator', value: '=', range: [Array] },
  ]
}
*/



### file extension to language



#### the code 

file extension to language?

ext to lang?

language from file extension?


In [ ]:
var path = require('path')
var ext_to_lang = {
    // extension:language
    'c': 'cpp',
    'h': 'cpp',
    'cpp': 'cpp',
    'dtd': 'dtd',
    'htm': 'html',
    'html': 'html',
    'java': 'java',
    'js': 'javascript',
    'mysql': 'mysql',
    'pl': 'perl',
    'php': 'php',
    'py': 'python',
    'rb': 'ruby',
    'sql': 'sql',
    'xml': 'xml',
    'vbs': 'vbscript',
    'as':'actionscript',
    'abap':'abap',
    'a':'ada',
    'applescript':'applescript',
    'x86':'asm',
    'asm':'asm',
    'm68k':'m68k',
    'pic16':'pic16',
    'z80':'z80',
    'log':'apache',
    'list':'apt_sources',
    'asp':'asp',
    'aspx':'asp',
    'aut':'autoit',
    'abnf':'bnf',
    'bnf':'bnf',
    'sh':'bash',
    'bash':'bash',
    'bgm':'basic4gl',
    'bb':'blitzbasic',
    'b':'bf',
    'bf':'bf',
    'c':'cpp',
    'h':'cpp',
    'cpp':'cpp',
    'cs':'csharp',
    'csharp':'csharp',
    'dcl':'caddcl',
    'lsp':'cadlisp',
    'cfdg':'cfdg',
    'cil':'cil',
    'cob':'cobol',
    'cbl':'cobol',
    'cobol':'cobol',
    'css':'css',
    'd':'d',
    'pas':'delphi',
    'dpr':'delphi',
    'patch':'diff',
    'diff':'diff',
    'batch':'dos',
    'dot':'dot',
    'eff':'eiffel',
    'f77':'fortran',
    'f95':'fortran',
    'ftn':'fortran',
    '4gl':'genero',
    'fbl':'freebasic',
    'mo':'gettext',
    'glsl':'glsl',
    'gml':'gml',
    'plt':'gnuplot',
    'groovy':'groovy',
    'gs':'haskell',
    'hs':'haskell',
    'hq9+':'hq9plus',
    'htm':'html',
    'html':'html',
    'ini':'ini',
    'ino':'inno',
    'myp':'inno',
    'i':'intercal',
    'io':'io',
    'java':'java',
    'js':'javascript',
    'javascript':'javascript',
    'kix':'kixtart',
    'tex':'latex',
    'lisp':'lisp',
    'lsp':'lisp',
    'lols':'lolcode',
    'lol':'lolcode',
    'lsx':'lotusscript',
    'lscript':'lscript',
    'lua':'lua',
    'makefile':'make',
    'make':'make',
    'mrc':'mirc',
    'mxml':'mxml',
    'mysql':'mysql',
    'nsh':'nsis',
    'nsi':'nsis',
    'cma':'ocaml',
    'p':'pascal',
    'pl':'perl',
    'php':'php',
    'php4':'php',
    'php5':'php',
    'phps':'php',
    'pbk':'pixelbender',
    'pl':'plsql',
    'pov':'povray',
    'ps':'powershell',
    'pl':'prolog',
    'pvx':'providex',
    'py':'python',
    'qbasic':'qbasic',
    'rb':'ruby',
    'ruby':'ruby',
    'sas':'sas',
    'sas':'scala',
    'scm':'scheme',
    'sce':'scilab',
    'st':'smalltalk',
    'sql':'sql',
    'tk':'tcl',
    'tcl':'tcl',
    'tclx':'tcl',
    'tbasic':'thinbasic',
    'typo3':'typoscript',
    'idl':'idl',
    'vb':'vbnet',
    'v':'verilog',
    'vhdl':'vhdl',
    'vim':'vim',
    'wbt':'winbatch',
    'reg':'reg',
    'xml':'xml',
    'x++':'xpp',
    'vbs':'vb',
}

var lang_to_parser = [
  '../Resources/Parsers/abnf/Abnf.g4',
  '../Resources/Parsers/agc/agc.g4',
  '../Resources/Parsers/algol60/algol60.g4',
  '../Resources/Parsers/alpaca/alpaca.g4',
  '../Resources/Parsers/antlr/antlr3/ANTLRv3.g4',
  '../Resources/Parsers/antlr/antlr4/ANTLRv4Lexer.g4',
  '../Resources/Parsers/antlr/antlr4/ANTLRv4LexerPythonTarget.g4',
  '../Resources/Parsers/antlr/antlr4/ANTLRv4Parser.g4',
  '../Resources/Parsers/antlr/antlr4/LexBasic.g4',
  '../Resources/Parsers/antlr/antlr4/examples/CPP14.g4',
  '../Resources/Parsers/antlr/antlr4/examples/Hello.g4',
  '../Resources/Parsers/antlr/antlr4/examples/Issue1567.g4',
  '../Resources/Parsers/antlr/antlr4/examples/issue1165.g4',
  '../Resources/Parsers/antlr/antlr4/examples/issue1165_2.g4',
  '../Resources/Parsers/antlr/antlr4/examples/issue1165_3.g4',
  '../Resources/Parsers/antlr/antlr4/examples/issue1165_4.g4',
  '../Resources/Parsers/antlr/antlr4/examples/three.g4',
  '../Resources/Parsers/antlr/antlr4/examples/twocomments.g4',
  '../Resources/Parsers/apex/apex.g4',
  '../Resources/Parsers/arithmetic/arithmetic.g4',
  '../Resources/Parsers/asm/asm6502/asm6502.g4',
  '../Resources/Parsers/asm/asm8080/asm8080.g4',
  '../Resources/Parsers/asm/asm8086/asm8086.g4',
  '../Resources/Parsers/asm/asmMASM/asmMASM.g4',
  '../Resources/Parsers/asm/asmZ80/asmZ80.g4',
  '../Resources/Parsers/asm/masm/MASM.g4',
  '../Resources/Parsers/asn/asn/ASN.g4',
  '../Resources/Parsers/asn/asn_3gpp/ASN_3gpp.g4',
  '../Resources/Parsers/aspectj/AspectJLexer.g4',
  '../Resources/Parsers/aspectj/AspectJParser.g4',
  '../Resources/Parsers/atl/ATL.g4',
  '../Resources/Parsers/b/b.g4',
  '../Resources/Parsers/basic/jvmBasic.g4',
  '../Resources/Parsers/bnf/bnf.g4',
  '../Resources/Parsers/brainfuck/brainfuck.g4',
  '../Resources/Parsers/c/C.g4',
  '../Resources/Parsers/calculator/calculator.g4',
  '../Resources/Parsers/capnproto/CapnProto.g4',
  '../Resources/Parsers/clf/clf.g4',
  '../Resources/Parsers/clif/CLIF.g4',
  '../Resources/Parsers/clojure/Clojure.g4',
  '../Resources/Parsers/clu/clu.g4',
  '../Resources/Parsers/cmake/CMake.g4',
  '../Resources/Parsers/cobol85/Cobol85.g4',
  '../Resources/Parsers/cobol85/Cobol85Preprocessor.g4',
  '../Resources/Parsers/cookie/cookie.g4',
  '../Resources/Parsers/cool/COOL.g4',
  '../Resources/Parsers/cpp/CPP14.g4',
  '../Resources/Parsers/cql3/CqlLexer.g4',
  '../Resources/Parsers/cql3/CqlParser.g4',
  '../Resources/Parsers/creole/creole.g4',
  '../Resources/Parsers/csharp/CSharpLexer.g4',
  '../Resources/Parsers/csharp/CSharpParser.g4',
  '../Resources/Parsers/csharp/CSharpPreprocessorParser.g4',
  '../Resources/Parsers/csharp/CSharpSharwell/CSharpLexer.g4',
  '../Resources/Parsers/csharp/CSharpSharwell/CSharpPreprocessorParser.g4',
  '../Resources/Parsers/css3/css3.g4',
  '../Resources/Parsers/csv/CSV.g4',
  '../Resources/Parsers/cto/CtoLexer.g4',
  '../Resources/Parsers/cto/CtoParser.g4',
  '../Resources/Parsers/dart2/Dart2.g4',
  '../Resources/Parsers/databank/databank.g4',
  '../Resources/Parsers/dcm/DCM_2_0_grammar.g4',
  '../Resources/Parsers/dice/DiceNotation.g4',
  '../Resources/Parsers/dice/DiceNotationLexer.g4',
  '../Resources/Parsers/dot/DOT.g4',
  '../Resources/Parsers/edif300/EDIF300.g4',
  '../Resources/Parsers/erlang/Erlang.g4',
  '../Resources/Parsers/fasta/fasta.g4',
  '../Resources/Parsers/fen/fen.g4',
  '../Resources/Parsers/flatbuffers/FlatBuffers.g4',
  '../Resources/Parsers/fol/fol.g4',
  '../Resources/Parsers/fortran77/Fortran77Lexer.g4',
  '../Resources/Parsers/fortran77/Fortran77Parser.g4',
  '../Resources/Parsers/fusion-tables/FusionTablesSql.g4',
  '../Resources/Parsers/gff3/gff3.g4',
  '../Resources/Parsers/gml/gml.g4',
  '../Resources/Parsers/golang/GoLexer.g4',
  '../Resources/Parsers/golang/GoParser.g4',
  '../Resources/Parsers/graphql/GraphQL.g4',
  '../Resources/Parsers/graphstream-dgs/DGSLexer.g4',
  '../Resources/Parsers/graphstream-dgs/DGSParser.g4',
  '../Resources/Parsers/gtin/gtin.g4',
  '../Resources/Parsers/guido/guido.g4',
  '../Resources/Parsers/html/HTMLLexer.g4',
  '../Resources/Parsers/html/HTMLParser.g4',
  '../Resources/Parsers/http/http.g4',
  '../Resources/Parsers/hypertalk/HyperTalk.g4',
  '../Resources/Parsers/icalendar/ICalendar.g4',
  '../Resources/Parsers/icon/icon.g4',
  '../Resources/Parsers/idl/IDL.g4',
  '../Resources/Parsers/inf/inf.g4',
  '../Resources/Parsers/informix/informix.g4',
  '../Resources/Parsers/iri/IRI.g4',
  '../Resources/Parsers/istc/istc.g4',
  '../Resources/Parsers/java/java/JavaLexer.g4',
  '../Resources/Parsers/java/java/JavaParser.g4',
  '../Resources/Parsers/java/java8/Java8Lexer.g4',
  '../Resources/Parsers/java/java8/Java8Parser.g4',
  '../Resources/Parsers/java/java9/Java9.g4',
  '../Resources/Parsers/javadoc/JavadocLexer.g4',
  '../Resources/Parsers/javadoc/JavadocParser.g4',
  '../Resources/Parsers/javascript/ecmascript/CSharp/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/CSharpSharwell/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/Go/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/JavaScript/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/Python/ECMAScript.g4',
  '../Resources/Parsers/javascript/javascript/JavaScriptLexer.g4',
  '../Resources/Parsers/javascript/javascript/JavaScriptParser.g4',
  '../Resources/Parsers/jpa/JPA.g4',
  '../Resources/Parsers/json/JSON.g4',
  '../Resources/Parsers/jsonquery/JsonQuery.g4',
  '../Resources/Parsers/kirikiri-tjs/TJSLexer.g4',
  '../Resources/Parsers/kirikiri-tjs/TJSParser.g4',
  '../Resources/Parsers/kotlin/kotlin/KotlinLexer.g4',
  '../Resources/Parsers/kotlin/kotlin/KotlinParser.g4',
  '../Resources/Parsers/kotlin/kotlin/UnicodeClasses.g4',
  '../Resources/Parsers/kotlin/kotlin-formal/KotlinLexer.g4',
  '../Resources/Parsers/kotlin/kotlin-formal/KotlinParser.g4',
  '../Resources/Parsers/kotlin/kotlin-formal/UnicodeClasses.g4',
  '../Resources/Parsers/kuka/krl.g4',
  '../Resources/Parsers/lambda/lambda.g4',
  '../Resources/Parsers/lcc/lcc.g4',
  '../Resources/Parsers/less/LessLexer.g4',
  '../Resources/Parsers/less/LessParser.g4',
  '../Resources/Parsers/logo/logo/logo.g4',
  '../Resources/Parsers/logo/ucb-logo/UCBLogo.g4',
  '../Resources/Parsers/lolcode/lolcode.g4',
  '../Resources/Parsers/lpc/LPC.g4',
  '../Resources/Parsers/lua/Lua.g4',
  '../Resources/Parsers/matlab/matlab.g4',
  '../Resources/Parsers/mdx/mdx.g4',
  '../Resources/Parsers/memcached_protocol/memcached_protocol.g4',
  '../Resources/Parsers/metric/metric.g4',
  '../Resources/Parsers/modelica/modelica.g4',
  '../Resources/Parsers/modula2pim4/m2pim4.g4',
  '../Resources/Parsers/molecule/molecule.g4',
  '../Resources/Parsers/moo/moo.g4',
  '../Resources/Parsers/morsecode/morsecode.g4',
  '../Resources/Parsers/mps/mps.g4',
  '../Resources/Parsers/muddb/muddb.g4',
  '../Resources/Parsers/mumath/mumath.g4',
  '../Resources/Parsers/mumps/mumps.g4',
  '../Resources/Parsers/muparser/MuParser.g4',
  '../Resources/Parsers/objc/ObjectiveCParser.g4',
  '../Resources/Parsers/objc/one-step-processing/ObjectiveCLexer.g4',
  '../Resources/Parsers/objc/one-step-processing/ObjectiveCPreprocessorParser.g4',
  '../Resources/Parsers/objc/two-step-processing/ObjectiveCLexer.g4',
  '../Resources/Parsers/objc/two-step-processing/ObjectiveCPreprocessorLexer.g4',
  '../Resources/Parsers/objc/two-step-processing/ObjectiveCPreprocessorParser.g4',
  '../Resources/Parsers/oncrpc/oncrpcv2.g4',
  '../Resources/Parsers/oncrpc/xdr.g4',
  '../Resources/Parsers/p/p.g4',
  '../Resources/Parsers/pascal/pascal.g4',
  '../Resources/Parsers/pcre/PCRE.g4',
  '../Resources/Parsers/pddl/Pddl.g4',
  '../Resources/Parsers/pdn/pdn.g4',
  '../Resources/Parsers/pdp7/pdp7.g4',
  '../Resources/Parsers/peoplecode/PeopleCode.g4',
  '../Resources/Parsers/pgn/PGN.g4',
  '../Resources/Parsers/php/PhpLexer.g4',
  '../Resources/Parsers/php/PhpParser.g4',
  '../Resources/Parsers/pike/pike.g4',
  '../Resources/Parsers/pl0/pl0.g4',
  '../Resources/Parsers/ply/ply.g4',
  '../Resources/Parsers/postalcode/postalcode.g4',
  '../Resources/Parsers/powerbuilder/PowerBuilderLexer.g4',
  '../Resources/Parsers/powerbuilder/PowerBuilderParser.g4',
  '../Resources/Parsers/prolog/prolog.g4',
  '../Resources/Parsers/propcalc/propcalc.g4',
  '../Resources/Parsers/properties/properties.g4',
  '../Resources/Parsers/protobuf3/Protobuf3.g4',
  '../Resources/Parsers/prov-n/PROV_N.g4',
  '../Resources/Parsers/python/python/PythonLexer.g4',
  '../Resources/Parsers/python/python/PythonParser.g4',
  '../Resources/Parsers/python/python2/Python2.g4',
  '../Resources/Parsers/python/python2-js/Python2.g4',
  '../Resources/Parsers/python/python3/Python3.g4',
  '../Resources/Parsers/python/python3-cs/Python3.g4',
  '../Resources/Parsers/python/python3-js/Python3.g4',
  '../Resources/Parsers/python/python3-py/Python3.g4',
  '../Resources/Parsers/python/python3-ts/Python3.g4',
  '../Resources/Parsers/python/python3_without_actions/Main/Python3.g4',
  '../Resources/Parsers/python/python3alt/AltPython3.g4',
  '../Resources/Parsers/quakemap/quakemap.g4',
  '../Resources/Parsers/r/R.g4',
  '../Resources/Parsers/r/RFilter.g4',
  '../Resources/Parsers/rcs/RCS.g4',
  '../Resources/Parsers/redcode/redcode.g4',
  '../Resources/Parsers/rego/RegoLexer.g4',
  '../Resources/Parsers/rego/RegoParser.g4',
  '../Resources/Parsers/restructuredtext/ReStructuredText.g4',
  '../Resources/Parsers/rexx/RexxLexer.g4',
  '../Resources/Parsers/rexx/RexxParser.g4',
  '../Resources/Parsers/rfc822/rfc822-datetime/datetime.g4',
  '../Resources/Parsers/rfc822/rfc822-emailaddress/emailaddress.g4',
  '../Resources/Parsers/robotwars/robotwar.g4',
  '../Resources/Parsers/romannumerals/romannumerals.g4',
  '../Resources/Parsers/rpn/rpn.g4',
  '../Resources/Parsers/ruby/Corundum.g4',
  '../Resources/Parsers/scala/Scala.g4',
  '../Resources/Parsers/scotty/scotty.g4',
  '../Resources/Parsers/scss/ScssLexer.g4',
  '../Resources/Parsers/scss/ScssParser.g4',
  '../Resources/Parsers/sexpression/sexpression.g4',
  '../Resources/Parsers/sgf/sgf.g4',
  '../Resources/Parsers/sharc/SHARCLexer.g4',
  '../Resources/Parsers/sharc/SHARCParser.g4',
  '../Resources/Parsers/sickbay/sickbay.g4',
  '../Resources/Parsers/smalltalk/Smalltalk.g4',
  '../Resources/Parsers/smiles/smiles.g4',
  '../Resources/Parsers/smtlibv2/SMTLIBv2.g4',
  '../Resources/Parsers/snobol/snobol.g4',
  '../Resources/Parsers/solidity/Solidity.g4',
  '../Resources/Parsers/sparql/Sparql.g4',
  '../Resources/Parsers/sql/mysql/Positive-Technologies/MySqlLexer.g4',
  '../Resources/Parsers/sql/mysql/Positive-Technologies/MySqlParser.g4',
  '../Resources/Parsers/sql/plsql/PlSqlLexer.g4',
  '../Resources/Parsers/sql/plsql/PlSqlParser.g4',
  '../Resources/Parsers/sql/sqlite/SQLite.g4',
  '../Resources/Parsers/sql/tsql/TSqlLexer.g4',
  '../Resources/Parsers/sql/tsql/TSqlParser.g4',
  '../Resources/Parsers/stacktrace/StackTrace.g4',
  '../Resources/Parsers/stellaris/stellaris.g4',
  '../Resources/Parsers/stringtemplate/LexBasic.g4',
  '../Resources/Parsers/stringtemplate/LexUnicode.g4',
  '../Resources/Parsers/stringtemplate/STGLexer.g4',
  '../Resources/Parsers/stringtemplate/STGParser.g4',
  '../Resources/Parsers/stringtemplate/STLexer.g4',
  '../Resources/Parsers/stringtemplate/STParser.g4',
  '../Resources/Parsers/suokif/SUOKIF.g4',
  '../Resources/Parsers/swift/swift2/Swift2.g4',
  '../Resources/Parsers/swift/swift3/Swift3.g4',
  '../Resources/Parsers/swift-fin/src/main/antlr4/SwiftFinLexer.g4',
  '../Resources/Parsers/swift-fin/src/main/antlr4/SwiftFinParser.g4',
  '../Resources/Parsers/telephone/telephone.g4',
  '../Resources/Parsers/thrift/Thrift.g4',
  '../Resources/Parsers/tiny/tiny.g4',
  '../Resources/Parsers/tinybasic/tinybasic.g4',
  '../Resources/Parsers/tinyc/tinyc.g4',
  '../Resources/Parsers/tinymud/tinymud.g4',
  '../Resources/Parsers/tnsnames/tnsnamesLexer.g4',
  '../Resources/Parsers/tnsnames/tnsnamesParser.g4',
  '../Resources/Parsers/tnt/tnt.g4',
  '../Resources/Parsers/toml/toml.g4',
  '../Resources/Parsers/tsv/tsv.g4',
  '../Resources/Parsers/turtle/TURTLE.g4',
  '../Resources/Parsers/turtle-doc/turtle.g4',
  '../Resources/Parsers/typescript/TypeScriptLexer.g4',
  '../Resources/Parsers/typescript/TypeScriptParser.g4',
  '../Resources/Parsers/unicode/graphemes/Graphemes.g4',
  '../Resources/Parsers/unicode/unicode16/classify.g4',
  '../Resources/Parsers/upnp/Upnp.g4',
  '../Resources/Parsers/url/url.g4',
  '../Resources/Parsers/useragent/useragent.g4',
  '../Resources/Parsers/v/V.g4',
  '../Resources/Parsers/vb6/VisualBasic6.g4',
  '../Resources/Parsers/vba/vba.g4',
  '../Resources/Parsers/verilog/system_verilog/SysVerilogHDL.g4',
  '../Resources/Parsers/verilog/verilog/Verilog2001.g4',
  '../Resources/Parsers/vhdl/vhdl.g4',
  '../Resources/Parsers/wat/WatLexer.g4',
  '../Resources/Parsers/wat/WatParser.g4',
  '../Resources/Parsers/wavefront/WavefrontOBJ.g4',
  '../Resources/Parsers/webidl/WebIDL.g4',
  '../Resources/Parsers/wkt/wkt.g4',
  '../Resources/Parsers/xml/XMLLexer.g4',
  '../Resources/Parsers/xml/XMLParser.g4',
  '../Resources/Parsers/xpath/xpath.g4',
  '../Resources/Parsers/xsd-regex/regexLexer.g4',
  '../Resources/Parsers/xsd-regex/regexParser.g4',
  '../Resources/Parsers/z/ZLexer.g4',
  '../Resources/Parsers/z/ZOperatorParser.g4',
  '../Resources/Parsers/z/ZParser.g4'
]

function extToLang(filename) {
    var result = ext_to_lang[filename]
        || filename.replace(/^\./, '')
        || ext_to_lang[path.extname(filename).replace(/^\./, '')]
    if(!result) throw new Error(`Unknown filetype ${filename}`)
    return result
}

module.exports = extToLang


### automatically generate all parsers

Because why the heck not?



#### the code

generate antlr parsers?


In [ ]:
var path = require('path')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var execCmd = importer.import('spawn child process')
var {chext} = importer.import('changing file name')

async function generateAntlr() {
    var files = glob('**/*.g4', path.join(__dirname, '../Resources/Parsers/'))

    /*
    for(var i = 0; i < files.length; i++) {
        try {
            await execCmd(`npx antlr4ts -visitor "${files[i]}"`)
        } catch (e) {
        }
        
        if(fs.existsSync(chext(files[i], '') + 'Parser.js')
           || fs.existsSync(chext(files[i], '.js'))) {
            continue
        }
        try {
            await execCmd(`java -Xmx500M -cp "/Users/briancullinan/Downloads/antlr-4.8-complete.jar:$CLASSPATH" org.antlr.v4.Tool -Dlanguage=JavaScript -visitor "${path.dirname(files[i])}"/*.g4`)
        } catch (e) {
        }
    }
    */
    
    var typescripts = glob('**/*.ts', path.join(__dirname, '../Resources/Parsers/'))
        .map(f => path.dirname(f))
        .filter((f, i, arr) => arr.indexOf(f) === i)
    
    for(var i = 0; i < typescripts.length; i++) {
        try {
            await execCmd(`npx babel --extensions .ts,.tsx "${typescripts[i]}" --out-dir "${typescripts[i]}"`)
        } catch (e) {
        }
    }
    
}

module.exports = generateAntlr
